In [8]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Conv2D, Dropout, MaxPool2D, Dense, Flatten, Input
from tensorflow.nn import local_response_normalization as LRN

### 구조
- Input: 224 * 224 * 3, 
- Conv2D 5층
    - 필터 수 96, 크기 11 * 11, stride 4
        - LRN(k = 2, n = 5, α = 10^(−4), and β = 0.75)
        - pool(stride = 2, filter size = 3)
    - 필터 수 256, 크기 5 * 5
        - LRN(k = 2, n = 5, α = 10^(−4), and β = 0.75)
        - pool(stride = 2, filter size = 3)
    - 필터 수 384, 크기 3 * 3
    - 필터 수 384, 크기 5 * 5
    - 필터 수 256, 크기 5 * 5
- FCN 3층
    - 4096, dropout(0.5)
    - 4096, dropout(0.5)
    - 1000
- weight initialization(N~(0, 0.01)): 2, 4, 5번 conv & FCN
- batch size = 128
- momentum = 0.9
- weight decay = 0.0005
- epochs 90
- lr = 0.01
- reduce lr on plateau: 3회, 0.1배

In [9]:
inputs = Input(shape=(224, 224, 3))

# Conv 1
conv1 = Conv2D(96, (11, 11), 
               strides=4, 
               activation='relu')(inputs)
lrn1 = LRN(conv1, bias=2, depth_radius=5, alpha=0.0001, beta= 0.75)
pool1 = MaxPool2D((3, 3), 2)(lrn1)

# Conv 2
conv2 = Conv2D(256, (5, 5), 
               strides=1, 
               activation='relu', 
               padding='same',
               kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01))(pool1)
lrn2 = LRN(conv2, bias=2, depth_radius=5, alpha=0.0001, beta= 0.75)
pool2 = MaxPool2D((3, 3), 2)(lrn2)

# Conv 3
conv3 = Conv2D(384, (3, 3), 
               strides=1, 
               activation='relu',
               padding='same')(pool2)

# Conv 4
conv4 = Conv2D(384, (5, 5), 
               strides=1, 
               activation='relu',
               padding='same')(conv3)

# Conv 5
conv5 = Conv2D(256, (5, 5), 
               strides=1, 
               activation='relu',
               padding='same')(conv4)
pool3 = MaxPool2D((3, 3), 2)(conv5)

# FCN 1
flat = Flatten()(pool3)
fcn1 = Dense(4096, activation='relu',
             kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01))(flat)
drop1 = Dropout(0.5)(fcn1)

# FCN 2
fcn2 = Dense(4096, activation='relu',
             kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01))(drop1)
drop2 = Dropout(0.5)(fcn2)

# FCN 3
fcn3 = Dense(1000, activation='softmax')(drop2)

model = Model(inputs=inputs, outputs=fcn3)

2023-01-06 21:38:50.987156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-06 21:38:50.987267: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-06 21:38:50.987331: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-06 21:38:50.987361: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-06 21:38:50.987392: W tensorf

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 tf.nn.local_response_normal  (None, 54, 54, 96)       0         
 ization (TFOpLambda)                                            
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 tf.nn.local_response_normal  (None, 26, 26, 256)      0     